# Adding a new model

RayLLM supports fine-tuned versions of models in the `models` directory as well as model architectures supported by [vLLM](https://docs.vllm.ai/en/latest/models/supported_models.html). You can either bring a model from HuggingFace or artifact storage like S3, GCS.

We generally recommend running the starter script via ``python main.py`` to import your own model. You can go through the below steps if your model isn't based on one of the provided model architectures and prefer to construct the model config yaml yourself. We have also provided some starter yaml files and you can edit the model to the specific model file that you created. Once you create the model YAML files, you will need to add these model YAMLs into a config file similar to `llm-serve-aws.yaml` that can deployed locally or to an Anyscale Service.



## Adding a private model

For loading a model from S3 or GCS, set `engine_config.s3_mirror_config.bucket_uri` or `engine_config.gcs_mirror_config.bucket_uri` to point to a folder containing your model and tokenizer files (`config.json`, `tokenizer_config.json`, `.bin`/`.safetensors` files, etc.) and set `engine_config.model_id` to any ID you desire in the `organization/model` format, eg. `myorganization/llama2-finetuned`. The model will be downloaded to a folder in the `<TRANSFORMERS_CACHE>/models--<organization-name>--<model-name>/snapshots/<HASH>` directory on each node in the cluster. `<HASH>` will be determined by the contents of `hash` file in the S3 folder, or default to `0000000000000000000000000000000000000000`. See the [HuggingFace transformers documentation](https://huggingface.co/docs/transformers/main/en/installation#cache-setup).

For loading a model from an accessible S3 bucket:

```yaml
engine_config:
  model_id: YOUR_MODEL_NAME
  s3_mirror_config:
    bucket_uri: s3://YOUR_BUCKET_NAME/YOUR_MODEL_FOLDER
    extra_files: []
```

For loading a model from an accessible Google Cloud Storage bucket:

```yaml
engine_config:
  model_id: YOUR_MODEL_NAME
  s3_mirror_config:
    bucket_uri: gs://YOUR_BUCKET_NAME/YOUR_MODEL_FOLDER
    extra_files: []
```



### Prompt Format
A prompt format is used to convert a chat completions API input into a prompt to feed into the LLM engine. The format is a dictionary where the key refers to one of the chat actors and the value is a string template for which to convert the content of the message into a string. Each message in the API input is formated into a string and these strings are assembled together to form the final prompt.

The string template should include the `{instruction}` keyword, which will be replaced with message content from the ChatCompletions API.

For example, if a user sends the following message for llama2-7b-chat-hf ([prompt format](models/llama/meta-llama--Llama-2-7b-chat-hf_a10g_tp1.yaml#L30-L36)):
```json
{
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant."
    },
    {
      "role": "user",
      "content": "What is the capital of France?"
    },
    {
      "role": "assistant",
      "content": "The capital of France is Paris."
    },
    {
      "role": "user",
      "content": "What about Germany?"
    }
  ]
}
```
The generated prompt that is sent to the LLM engine will be:
```
[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

What is the capital of France? [/INST] The capital of France is Paris. </s><s>[INST] What about Germany? [/INST]
```

#### Schema

The following keys are supported:
* `system` - The system message. This is a message inserted at the beginning of the prompt to provide instructions for the LLM.
* `assistant` - The assistant message. These messages are from the past turns of the assistant as defined in the list of messages provided in the ChatCompletions API.
* `trailing_assistant` - The special characters that will be added to the end of the prompt before sending it to the LLM for generation. This often includes special characters that put the LLM into assitant mode (granted that model has been trained to support such special keywords). For example [vicuna](https://huggingface.co/TheBloke/vicuna-13B-v1.5-GGML) has `Assistant: ` as a special tag that can go here.
* `user` - The user message. This is the messages of the user as defined in the list of messages provided in the ChatCompletions API.

In addition, there some configurations to control the prompt formatting behavior:
* `default_system_message` - The default system message. This system message is used by default if one is not provided in the ChatCompletions API.
* `system_in_user` - Whether the system prompt should be included in the user prompt. If true, the user field should include '{system}'.
* `add_system_tags_even_if_message_is_empty` - Whether to include the system tags even if the user message is empty.
* `strip_whitespace` - Whether to automatically strip whitespace from left and right of the content for the messages provided in the ChatCompletions API.


#### Example prompt config (Llama-based model)

```
prompt_format:
  system: "<<SYS>>\n{instruction}\n<</SYS>>\n\n"
  assistant: " {instruction} </s><s>"
  trailing_assistant: ""
  user: "[INST] {system}{instruction} [/INST]"
  system_in_user: true
  default_system_message: ""
stopping_sequences: []
```



#### Example prompt config (Mistral-based-model)

```
prompt_format:
  system: "<<SYS>>\n{instruction}\n<</SYS>>\n\n"
  assistant: " {instruction} </s><s> "
  trailing_assistant: " "
  user: "[INST] {system}{instruction} [/INST]"
  system_in_user: true
  default_system_message: "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
stopping_sequences: []
```



#### Example prompt config (Falcon-based-model)

```
prompt_format:
  system: "<|prefix_begin|>{instruction}<|prefix_end|>"
  assistant: "<|assistant|>{instruction}<|endoftext|>"
  trailing_assistant: "<|assistant|>"
  user: "<|prompter|>{instruction}<|endoftext|>"
  default_system_message: "Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant is happy to help with almost anything, and will do its best to understand exactly what is needed. It also tries to avoid giving false or misleading information, and it caveats when it isn't entirely sure about the right answer. That said, the assistant is practical and really does its best, and doesn't let caution get too much in the way of being useful."
stopping_sequences: ["<|prompter|>", "<|assistant|>", "<|endoftext|>"]
```
